In [1]:
import sys
sys.path.append("/home/kieran/code/Ant-mel/legendary_game_recs/")

from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *

In [2]:
raw_data = pd.read_csv("../raw_data/reference_csv_v1.csv", low_memory=False)

In [3]:
reference_data = make_reference_df_full_csv(raw_data)

In [4]:
reference_data

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3,link,game_id
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,[Melancholy Marionette],2.2,[Visual Novel],"[Windows PC, Mac, Linux]",...,0,0,0,0,0,0,0,1,/games/darling-duality-winter-wish/,178780.0
1,Anger Foot,2024-12-31,80,1,82,297,"[Devolver Digital, Robbie Fraser]",3.3,"[Brawler, Indie, Shooter]",[Windows PC],...,1,0,0,0,0,0,0,1,/games/anger-foot/,147184.0
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,[Long Tale Games],2.4,"[Adventure, RPG, Simulator]",[Windows PC],...,0,0,0,0,0,0,0,1,/games/life-is-feudal-mmo/,24611.0
3,The Making of Karateka,2023-08-29,10,3,9,41,[Digital Eclipse],4.5,"[Adventure, Fighting, Platform]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,0,0,1,0,0,0,0,1,/games/the-making-of-karateka/,256293.0
4,Enigma of Fear,2023-12-31,16,0,26,155,[Dumativa Game Studio],2.1,"[Adventure, Indie, Puzzle]",[Windows PC],...,0,0,0,0,0,0,0,1,/games/enigma-of-fear/,140133.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,Final Crisis: Terrestrial Defense Police,1991-12-01,4,0,1,4,[Techno Grard],3.8,[Shooter],[PC-8801],...,1,0,0,0,0,1,0,0,/games/final-crisis/,141644.0
36846,Level Up!,2009-11-25,6,0,0,1,[],2.7,"[Adventure, Indie]",[Web browser],...,0,0,0,0,0,0,1,0,/games/level-up--1/,95373.0
36847,Sea Battle,2014-02-07,3,0,0,0,[BYRIL],2.2,[],"[Android, iOS]",...,0,0,0,0,0,0,0,1,/games/sea-battle--2/,134381.0
36848,We Are Illuminati,2017-11-02,18,0,0,0,[Tapps Games],2.0,"[Indie, Simulator, Strategy]","[Android, iOS]",...,0,1,0,0,0,0,0,1,/games/we-are-illuminati/,120333.0


In [5]:
reference_data['description'] = reference_data['description'].astype(str)

In [6]:
extended_stop_words = ['able','access','across','also','always','another','away','back','become','best','better','big','box','bring','certain','clear','close','come','console','content','could','course','digital','dont','one','two','three','four','five','six','seven','eight','nine','ten','hundred','thousand','either','enjoy','enough','even','exclusive','extra','feature','franchise','full','fully','fun','game','gameplay','genre','get','give','go','good','great','great','greatest','happen','however','huge','ii','improve','include','increase','inside','interactive','introduce','instead','involve','large','last','later','launch','lead','let','level','like','little','look','main','meet','might','mix','modern','mode','must','name','new','next','need','number','nintendo','official','offer','object','option','order','original','originally','others','part','pc','perfect','platform','playable','player','playstation','plus','possible','port','prepare','previous','progress','project','put','reach','ready','remain','return','screen','scroll','second','first','third','see','sega','sequel','series','set','several','show','side','similar','since','small','something','sound','special','start','stat','state','stay','still','studio','super','take','switch','tell','test','th','though','throughout','title','together','try','ultimate','unique','update','upon','us','use','version','via','wait','want','wii','within','without','would','xbox','youll','youre','youve']

In [7]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize 
    stop_words = stopwords.words('english') ## define stopwords
    stop_words.extend(extended_stop_words)
    
    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    
    noun_lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "n") 
        for word in tokenized_sentence_cleaned
    ]
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in noun_lemmatized
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence

In [8]:
reference_data['clean_text'] = reference_data['description'].apply(preprocessing)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.02)

vectorized_descriptions = vectorizer.fit_transform(reference_data['clean_text'])
vectorized_descriptions = pd.DataFrame(
    vectorized_descriptions.toarray(), 
    columns = vectorizer.get_feature_names_out()
)

vectorized_descriptions

,ability,action,add,advance,adventure,alien,allow,along,ancient,arcade,...,visual,war,way,weapon,well,win,work,world,year,young
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
2,0.257052,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.252447,0.0,0.0,0.164194,0.00000,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.57374,0.0
36846,0.265993,0.0,0.000000,0.0,0.0,0.0,0.0,0.282995,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
36847,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.150856,0.00000,0.0
36848,0.000000,0.0,0.612331,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.433409,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0


In [10]:
from sklearn.decomposition import LatentDirichletAllocation

# Instantiate the LDA 

lda_model = LatentDirichletAllocation(n_components=10, max_iter = 50, learning_method='online',)

# Fit the LDA on the vectorized documents
lda_model.fit(vectorized_descriptions)

LatentDirichletAllocation(learning_method='online', max_iter=50)

In [11]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [12]:
print_topics(lda_model, vectorizer)

Topic 0:
[('game', 433.7499281517664), ('arcade', 429.8261545921896), ('horror', 399.6100250062743), ('fighter', 316.24716241811956), ('japanese', 287.82672718160836), ('travel', 287.28214509679395), ('classic', 281.6698536173209), ('rpg', 280.58587078793914), ('fight', 248.91207751808597), ('character', 241.28243048220912)]
Topic 1:
[('ever', 330.7682422504365), ('dream', 306.2654309886834), ('family', 282.3862167827683), ('real', 264.8981683842027), ('land', 263.6759739301524), ('fly', 258.835761163375), ('rescue', 241.34271558065552), ('set', 210.89442070000206), ('make', 191.06266971956362), ('light', 179.35996324734924)]
Topic 2:
[('player', 972.5395271538756), ('battle', 770.8906884605932), ('enemy', 759.6737607274805), ('play', 734.2959636940935), ('character', 651.2152417421224), ('weapon', 583.4480046859794), ('world', 567.7271213021404), ('team', 551.1379429050877), ('shoot', 534.2874353566256), ('fight', 528.4888389448961)]
Topic 3:
[('nan', 874.9158607595162), ('race', 788.

In [ ]:
reference_data[['clean_text']]

In [ ]:
vectorized_example = reference_data['clean_text'][36819]

In [ ]:
vectorized_example

In [ ]:
ve2 = vectorizer.transform(pd.Series(vectorized_example))
ve2

In [ ]:
vec_array = lda_model.transform(ve2)

In [ ]:
vec_array[0].tolist()

In [ ]:
vec_array[0].tolist().index(max(vec_array[0].tolist()))

In [13]:
def nlp_topic(x):
    vectorized_x = vectorizer.transform(pd.Series(x))
    vec_x_array = lda_model.transform(vectorized_x)
    vec_list = vec_x_array[0].tolist()
    return vec_list.index(max(vec_list))
    

In [14]:
reference_data['topic'] = reference_data['clean_text'].apply(nlp_topic)

/home/kieran/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(


In [15]:
reference_data[['topic']].value_counts()

topic
2        14228
9         4843
6         3892
0         3341
4         2624
5         2462
7         1712
3         1339
1         1314
8         1095
dtype: int64

In [16]:
reference_data[['topic']]

,topic
0,0
1,1
2,2
3,5
4,4
...,...
36845,9
36846,9
36847,2
36848,2


In [17]:
pd.get_dummies(reference_data['topic'])

,0,1,2,3,4,5,6,7,8,9
0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
36845,0,0,0,0,0,0,0,0,0,1
36846,0,0,0,0,0,0,0,0,0,1
36847,0,0,1,0,0,0,0,0,0,0
36848,0,0,1,0,0,0,0,0,0,0


In [18]:
topics_for_36k_games = pd.get_dummies(reference_data['topic'])

In [19]:
topics_for_36k_games.to_csv("../raw_data/topics_v2.csv", index=False)